In [ ]:
!pip install -q optimum[onnxruntime]==1.23.3 transformers==4.46.3 torch==2.2.2+cu121
import time
import requests

In [ ]:
## we need some preprocessing functions from the model

In [ ]:
from optimum.onnxruntime import ORTModelForImageClassification
from transformers import AutoFeatureExtractor

# Load and export the model to ONNX
model = ORTModelForImageClassification.from_pretrained("rh-ai-bu/wildfire01", export=True)

# Load the feature extractor
feature_extractor = AutoFeatureExtractor.from_pretrained("rh-ai-bu/wildfire01")

# Create an ONNX-based pipeline
from optimum.pipelines import pipeline
onnx_pipeline = pipeline("image-classification", model=model, feature_extractor=feature_extractor, accelerator="ort")

In [ ]:
onnx_pipeline.save_pretrained("onnx_pipeline")

In [ ]:
# image = "pic2.jpg"
# Import the function from the script
from get_random_image import get_random_image





In [ ]:
## figure out the precise URL we need to hit:

import subprocess

def run_command(command):
    """Run an OS command and return its output, handling errors."""
    try:
        result = subprocess.check_output(command, shell=True, text=True).strip()
        return result
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while running the command '{command}': {e}")
        return None

# Commands to run
get_IS_name = "oc get inferenceservice -o jsonpath='{.items[0].metadata.name}'"
get_IS_URL = "oc get inferenceservice -o jsonpath='{.items[0].status.url}'"

# Retrieve results
deployed_model_name = run_command(get_IS_name)
infer_endpoint = run_command(get_IS_URL)

# Print the results
if deployed_model_name:
    print("Inference Service Name:", deployed_model_name)
else:
    print("Failed to retrieve Inference Service Name.")

if infer_endpoint:
    print("Inference Service URL:", infer_endpoint)
else:
    print("Failed to retrieve Inference Service URL.")


In [ ]:
infer_url = f"{infer_endpoint}/v2/models/{deployed_model_name}/infer"
print(infer_url)

In [ ]:
def rest_request(data,shape):
    json_data = {
        "inputs": [
            {
                "name": "pixel_values",
                "shape": shape,
                "datatype": "FP32",
                "data": data
            }
        ]
    }

    start_time = time.time()  # Record the start time
    response = requests.post(infer_url, json=json_data, verify=True)
    end_time = time.time()  # Record the end time

    # Calculate and print the response time
    response_time = end_time - start_time
    print(f"Response time: {response_time:.6f} seconds")

    return response

In [ ]:
image = get_random_image()
print(image)
img_preprocessed = onnx_pipeline.preprocess(image)
img_preprocessed["pixel_values"].shape

In [ ]:
# print(img_preprocessed)

prediction = rest_request(img_preprocessed["pixel_values"].tolist(),img_preprocessed["pixel_values"].shape)
# prediction


# prediction = rest_request(image_preprocessed["pixel_values"].tolist())



# 

In [ ]:
# prediction

In [ ]:
# prediction.json()

In [ ]:
def get_predict(max_iter):
    for i in range(1, max_iter + 1):
        print("Image "+str(i))
        image = get_random_image()
        img_preprocessed = onnx_pipeline.preprocess(image)

        prediction = rest_request(img_preprocessed["pixel_values"].tolist(),img_preprocessed["pixel_values"].shape)


get_predict(10)

In [ ]:
import time


def get_predict(max_iter):
    start_time = time.time()
    for i in range(1, max_iter + 1):
        print("Image " + str(i))
        image = get_random_image()
        img_preprocessed = onnx_pipeline.preprocess(image)

        prediction = rest_request(img_preprocessed["pixel_values"].tolist(), img_preprocessed["pixel_values"].shape)

        # Check if 60 seconds have passed
        if time.time() - start_time > 60:
            break

# Call the function with a large number, it will stop after 60 seconds
get_predict(1000000)


In [ ]:


import time
import numpy as np
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

# Load the ONNX model pipeline (assuming this is already set up as in your original code)
from optimum.onnxruntime import ORTModelForImageClassification
from transformers import AutoFeatureExtractor
from optimum.pipelines import pipeline
from get_random_image import get_random_image

# Load and export the model to ONNX
model = ORTModelForImageClassification.from_pretrained("rh-ai-bu/wildfire01", export=True)
feature_extractor = AutoFeatureExtractor.from_pretrained("rh-ai-bu/wildfire01")
onnx_pipeline = pipeline("image-classification", model=model, feature_extractor=feature_extractor, accelerator="ort")
onnx_pipeline.save_pretrained("onnx_pipeline")

# Set up the endpoint
deployed_model_name = "wildfire01"
infer_endpoint = "https://wildfire01-user1-ai-mazing.apps.prod.rhoai.rh-aiservices-bu.com"
infer_url = f"{infer_endpoint}/v2/models/{deployed_model_name}/infer"

# Function to send a single REST request
def rest_request(data):
    json_data = {
        "inputs": [
            {
                "name": "pixel_values",
                "shape": data.shape,
                "datatype": "FP32",
                "data": data.tolist()
            }
        ]
    }

    start_time = time.time()
    response = requests.post(infer_url, json=json_data, verify=True)
    end_time = time.time()

    response_time = end_time - start_time
    print(f"Response time: {response_time:.6f} seconds")

    return response

# Preprocess the image
image = get_random_image()
print(f"Selected image: {image}")
image_preprocessed = onnx_pipeline.preprocess(image)
pixel_values = image_preprocessed["pixel_values"]

# Function to handle a single prediction request
def process_request():
    response = rest_request(pixel_values)
    if response.status_code == 200:
        prediction_data = response.json()
        logits = prediction_data['outputs'][0]['data']
        scores = softmax(logits)
        return scores
    else:
        print(f"Request failed with status code: {response.status_code}")
        return None


# Softmax function
def softmax(logits):
    exp_logits = np.exp(logits - np.max(logits))
    return exp_logits / np.sum(exp_logits)

# Send 100 simultaneous requests using ThreadPoolExecutor
def send_simultaneous_requests():
    results = []
    with ThreadPoolExecutor(max_workers=100) as executor:
        futures = [executor.submit(process_request) for _ in range(100)]
        for future in as_completed(futures):
            try:
                result = future.result()
                if result is not None:  # Correctly check for None instead of relying on NumPy truth value
                    results.append(result)
            except Exception as e:
                print(f"Error during request processing: {e}")
    return results

# Execute the simultaneous requests
if __name__ == "__main__":
    all_results = send_simultaneous_requests()
    # print("All results:", all_results)